In [ ]:
#IMPORTANT: This is a handmade model for learning purposes only.
#However, it is possible to create your own predictive model without a fancy algorithm :)

#coin flip model with random 1/survived 0/died

#iterate over dataFrame rows as (index, Series) pairs: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.iterrows.html
for index, row in data1.iterrows():
  if random.random() > .5:
    data1.set_value(index, 'Random_Predict', 1)
  else:
    data1.set_value(index, 'Random_predict', 0)
    

#score random guess of survival. Use shortcut 1 = Right Guess and 0 = Wrong Guess
#the mean of the column will then equal the accuracy
data1['Random_Score'] = 0
data1.loc[(data1['Survived']==data1['Random_Predict']), 'Random_Score']=1
print('Coin Flip Model Accuracy : {:.2f}%'.format(data1['Random_Score'].mean()*100)


#we can also use scikit's accuracy_score function to save us a few lines of code
#http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score
print('Coin Flip Model Accuracy w/SciKit: {:.2f}%'.format(metrics.accuracy_score(data1['Survived'], data1['Random_Predict'])*100))

In [ ]:
#group by or pivot table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html
pivot_female = data1[data1.Sex=='female'].groupby(['Sex', 'Pclass', 'Embarked', 'FareBin'])['Survived'].mean()
print('Survival Decision Tree w/Female Node: \n', pivot_female)

pivot_male = data1[data1.Sex=='male'].groupby(['Sex', 'Title'])['Survived'].mean()
print('\n\nSurvival Decision Tree w/Male Node: \n', pivot_male)

In [ ]:
#handmade data model using brain power (and Microsoft Excel Pivot Tables for quick calculations)
def mytree(df):
  #initialize table to store predictions
  Model = pd.DataFrame(data={'Predict':[]})
  male_title = ['Master']

  for index, row in df.iterrows():

    #Question 1: Were you on the Titanic; majority died
    Model.loc[index, 'Predict']=0

    #Question 2: Are you female; majority survived
    if (df.loc[index, 'Sex']=='female'):
      Model.loc[index, 'Predict']=1

    #Question 3A Female - Class and Question 4 Embarked gain minimum information

    #Question 5B Female - FareBin; set anythin less than .5 in female node decision tree back to 0
    if((df.loc[index, 'Sex']=='female') &
       (df.loc[index, 'Pclass']==3)&
       (df.loc[index, 'Embarked']=='S') &
       (df.loc[index, 'Fare'] > 8)
       ):
      Model.loc[index, 'Predict'] = 0

    #Question 3B Male: Title; set anythin greater than .5 to 1 for majority survived
    if ((df.loc[index, 'Sex']=='male')&
        (df.loc[index, 'Title'] in male_title)
        ):
      Model.loc[index, 'Predict'] =1


  return Model


#model data
Tree_Predict = mytree(data1)
print('Decision Tree Model Accuracy/Precision Score: {:.2f%\n'.format(metrics.accuracy_score(data1['Survived'], Tree_Predict)*100))




#Accuracy Summary Report with http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report
#Where recall score = (true positives)/(true positive + false negative) w/1 being best:http://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score
#And F1 score = weighted average of precision and recall w/1 being best: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
print(metrics.classification_report(data1['Survived'], Tree_Predict))

In [ ]:
#Plot Accuracy Summary
#Credit: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues)

  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting 'normalize=True'.
  """

  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Normalized confusion matrix")
  else:
    print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() /2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i,j], fmt),
             horizontalalignment='center',
             color='white' if cm[i, j] > thresh else 'black')
    
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')


# Compute confusion matrix
cnf_matrix = metrics.confusion_matrix(data1['Survived'], Tree_Predict)
np.set_pritoptions(precision=2)

class_names = ['Dead', 'Survived']

#Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classess=class_names,
                      title= 'Confusion matrix, without normalization')

#Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classess=class_names, normalize=True, title='Normalized confusion matrix')

